## Setting Up

In [2]:
import os

MODEL = os.getenv("OPENAI_MODEL")

In [3]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=MODEL
)

## Agent Runner

Running the Agent Asynchronously

`Runner.run()` runs the agent in async mode

In [6]:
result = await Runner.run(
    agent,
    "Tell me a short story"
)

result.final_output

'Once in a small village, nestled between towering mountains, lived a young girl named Lila. She had a special gift—she could talk to animals. Every morning, she would stroll through the fields, sharing secrets with the rabbits, learning wisdom from the wise old owl, and laughing with the playful squirrels.\n\nOne day, a fierce storm swept through the village, causing a mudslide that blocked the main road. The villagers were scared, as it isolated them from the outside world. Lila, sensing the animals’ distress, gathered her furry friends. Together, they devised a plan. The rabbits would dig a path, while the strong deer would help clear away debris.\n\nAs they worked tirelessly, Lila encouraged them with her gentle words. After hours of teamwork, they finally opened a small passage. The villagers couldn’t believe their eyes when they saw Lila and her animal friends emerge. With joy and gratitude, they cheered for her bravery.\n\nThe next day, the villagers organized a feast to celebra

### Async Streaming Mode

Start by creating a response using `Runner.run_streamed(...)`  \
Then iterate over it asynchronously

In [7]:
response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_67ed7db490448191a7f60242bc1dd76b099562524aee8de2', created_at=1743617460.0, error=None, incomplete_details=None, instructions="You're a helpful assistant", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=False, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reaso

## Events

The streaming response returns **Events**. This helps us filter the output to extract relevant information.

There are several events:
* RawEvent: Token output
* AgentUpdated: Agent creation
* StreamEvent: A message being completed
* 

In [8]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
     if event.type == "raw_response_event" and \
         isinstance(event.data, ResponseTextDeltaEvent):
             print(event.data.delta, end="", flush=True)

Once upon a time in a quaint village, tucked between rolling hills, lived a curious little fox named Finn. Unlike other foxes who prowled the woods, Finn was enchanted by the stars. Each night, he would climb to the highest hill and gaze up at the sparkling sky, dreaming of adventures beyond his forest.

One evening, as Finn lay among the soft grass, he spotted a twinkling star that seemed to beckon him. Intrigued, he followed a winding path deeper into the woods. To his surprise, he stumbled upon a hidden glade where the air was thick with magic. In the center stood an ancient tree, its branches shimmering with starlight.

"Welcome, Finn," said the tree with a voice like rustling leaves. "You have come to seek your adventure."

Finn's eyes widened in awe. "Can you send me among the stars?"

With a gentle sway, the tree answered, "Close your eyes and believe. Your heart will guide you."

Taking a deep breath, Finn did as instructed. Suddenly, he felt weightless, and when he opened his 

## Tools

In [9]:
from agents import function_tool

@function_tool
def multiply(x: int, y: int) -> int:
    """Multiply 2 numbers to provide the product"""
    return x*y

In [10]:
agent = Agent(
    name="Assistant",
    instructions=(
        "You're a helpful assistant who uses",
        "provided tools to help whenever possible.",
        "Do not rely on your own knowledge too much,",
        "instead use the tools to answer queries."
    ),
    model=MODEL,
    tools=[multiply]
)

In [12]:
response = Runner.run_streamed(
    agent,
    "What is the product of 4 and 100 ?"
)

async for event in response.stream_events():
    print(event)

Instructions must be a string or a function, got ("You're a helpful assistant who uses", 'provided tools to help whenever possible.', 'Do not rely on your own knowledge too much,', 'instead use the tools to answer queries.')


AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions=("You're a helpful assistant who uses", 'provided tools to help whenever possible.', 'Do not rely on your own knowledge too much,', 'instead use the tools to answer queries.'), handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='multiply', description='Multiply 2 numbers to provide the product', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}, 'required': ['x', 'y'], 'title': 'multiply_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d6ef3c3f400>, strict_json_schema=True)], mcp_servers=[], input_guardrails=[], output_guardrails

Instructions must be a string or a function, got ("You're a helpful assistant who uses", 'provided tools to help whenever possible.', 'Do not rely on your own knowledge too much,', 'instead use the tools to answer queries.')


RawResponsesStreamEvent(data=ResponseOutputItemDoneEvent(item=ResponseFunctionToolCall(arguments='{"x":4,"y":100}', call_id='call_MqPg924Qx1jjvyzHDSpmYmy1', name='multiply', type='function_call', id='fc_67ed8019cbb481918b35cd8482e0ba630a7d40eb6acf0a77', status='completed'), output_index=0, type='response.output_item.done'), type='raw_response_event')
RawResponsesStreamEvent(data=ResponseCompletedEvent(response=Response(id='resp_67ed8018c5808191bd12236f4329df2f0a7d40eb6acf0a77', created_at=1743618072.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"x":4,"y":100}', call_id='call_MqPg924Qx1jjvyzHDSpmYmy1', name='multiply', type='function_call', id='fc_67ed8019cbb481918b35cd8482e0ba630a7d40eb6acf0a77', status='completed')], parallel_tool_calls=False, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='multiply', parameters={'properties': {'x': {'title': 'X', 't